In [1]:
import sys, os
import numpy as np
sys.path.append('./overlap-code')

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import patches
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree, datasets
from sklearn.neighbors import KernelDensity
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, f1_score, balanced_accuracy_score, precision_recall_curve, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from overrule.overrule import OverRule2Stage, OverRule
from overrule.baselines import knn, marginal, propscore, svm
from overrule.support import SVMSupportEstimator, SupportEstimator
from overrule.overlap import SupportOverlapEstimator
from overrule.ruleset import BCSRulesetEstimator, RulesetEstimator

from utils import get_data, rule_str

In [3]:
SEED=0

CNF=False
VERBOSE=True

ALPHA_s=0.99
N_REF_MULT_s=0.3
ALPHA_o=0.90
N_REF_MULT_o=0

LAMBDA0_s=0.1 # | 1e-05 fixed cost of term, the smaller the more rules you'll allow 
LAMBDA1_s=0.001 # cost per literal

LAMBDA0_o=0.0000001 # fixed cost of term, the smaller the more rules you'll allow 
LAMBDA1_o=0.0000001 # cost per literal

D=10  # Maximum extra rules per beam seach iteration
K=10  # Maximum results returned during beam search
B=28  # Width of Beam Search

np.random.seed(SEED)
w_eps = 1e-8
CAT_COLS = []


In [4]:
X_df, a, y = get_data()
X_df.shape

X_df_sample = X_df.sample(frac=1, axis=1)
# X_df_sample = X_df[X_df['v312_3'] == 1].sample(frac=1, axis=1)

a_sample = a.iloc[X_df_sample.index]

In [5]:
X_df_sample.shape

(5649, 256)

In [ ]:
f_cols = X_df_sample.columns

base_estimator = LogisticRegression(solver='liblinear', max_iter=2000, C=0.005)
learner = CalibratedClassifierCV(base_estimator=base_estimator, cv=5, method='isotonic')

O = propscore.PropensityOverlapEstimator(estimator=learner)

RS_s = BCSRulesetEstimator(n_ref_multiplier=N_REF_MULT_s, alpha=ALPHA_s, lambda0=LAMBDA0_s, lambda1=LAMBDA1_s, B=B, CNF=CNF, 
                           cat_cols=CAT_COLS, seed=SEED, K=K, D=D, binarizer='default')
RS_o = BCSRulesetEstimator(n_ref_multiplier=N_REF_MULT_o, alpha=ALPHA_o, lambda0=LAMBDA0_o, lambda1=LAMBDA1_o, B=B, CNF=CNF, 
                           cat_cols=CAT_COLS, seed=SEED, binarizer='default')

RS_s.fit(X_df_sample, a_sample)
M = OverRule2Stage(O, RS_o, RS_s, refit_s=False)
M.fit(X_df_sample, a_sample)

rules1 = M.rules(as_str=False)

In [9]:
ALPHA_s, LAMBDA0_s, LAMBDA1_s, LAMBDA0_o, LAMBDA1_o

(0.99, 0.9, 0.0001, 0.1, 1e-07)

In [ ]:
TPR = RS_s.predict(X_df_sample).mean()
FPR = RS_s.relative_volume
print(TPR, FPR)
print(1/2 -  (FPR)/2 + TPR/2)
print(M.score_vs_base(X_df_sample))

In [6]:
O2 = knn.KNNOverlapEstimator(k=30)

RS_s = BCSRulesetEstimator(n_ref_multiplier=N_REF_MULT_s, alpha=ALPHA_s, lambda0=LAMBDA0_s, lambda1=LAMBDA1_s, B=B, CNF=CNF, 
                           cat_cols=CAT_COLS, seed=SEED, K=K, D=D, binarizer='tree')

RS_o2 = BCSRulesetEstimator(n_ref_multiplier=N_REF_MULT_o, alpha=ALPHA_o, lambda0=LAMBDA0_o, lambda1=LAMBDA1_o, B=B, CNF=CNF, 
                           cat_cols=CAT_COLS, seed=SEED, binarizer='tree')

RS_s.fit(X_df_sample, a_sample)
M2 = OverRule2Stage(O2, RS_o2, RS_s, refit_s=False, bb_on_support=False)
M2.fit(X_df_sample, a_sample)

rules2 = M2.rules(as_str=False)

(439492, 28)
(5263, 6)


In [7]:
TPR = RS_s.predict(X_df_sample).mean()
FPR = RS_s.relative_volume
print(TPR, FPR)
print(1/2 -  (FPR)/2 + TPR/2)
print(M2.score_vs_base(X_df_sample))

0.9316693220038945 0.25482720707721457
0.83842105746334
0.9634861407249466


In [8]:
print(RS_o2.complexity())
rule_str(RS_o2.rules())

(1, 2)


'  ((X_df["v191"] <= 290545.000) & (X_df["v191"] > -188796.000))'

In [9]:
print(RS_s.complexity())
rule_str(RS_s.rules())

(2, 9)


'  (~X_df["v116_97"].astype(bool) & ~X_df["v122_7"].astype(bool) & (X_df["v201"] > 0.500))| (X_df["v116_97"].astype(bool) & X_df["v122_7"].astype(bool) & (X_df["v201"] > 0.500) & (X_df["v202"] <= 2.500) & ~X_df["v513_3"].astype(bool) & ~X_df["v602_4"].astype(bool))'

In [ ]:
rules = rules1

In [ ]:
print('Number of reference samples: {}'.format(RS_s.refSamples.shape[0]))
print('Coverage of data points: %.3f, Requested >= %.3f' % (TPR, RS_s.M.alpha))
print('Coverage of reference points: %.3f' % FPR)
print('Rules: {}'.format(rules))
print('Number of Rules: {}'.format(np.sum([len(rule) for rule in rules])))
print('Number of Literals: {}'.format(np.sum([len(rule_) for rule in rules for rule_ in rule])))
print('Complexity: {}'.format(RS_s.complexity()))
print('AUC between rules and base estimator: {}'.format(M2.score_vs_base(X_df_sample)))

import time
outfile = open('results.txt', 'a+')


outfile.write('Time: {}\n'.format(time.time()))
outfile.write('Binarizer: {}, {}, {}\n'.format(RS_s.binarizer, RS_o.binarizer, RS_o2.binarizer))
outfile.write('Params (knn): N_REF_MULT_s {}, N_REF_MULT_o {}, ALPHA_s {}, ALPHA_o {}, LAMBDA0_s {}, \
                LAMBDA1_s {}, LAMBDA0_o {}, LAMBDA1_o {}, B {}, CNF {}\n\n'.
                    format(N_REF_MULT_s, N_REF_MULT_o, ALPHA_s, ALPHA_o, LAMBDA0_s, LAMBDA1_s, LAMBDA0_o, LAMBDA1_o, B, CNF))

outfile.write('Number of reference samples: {}\n'.format(RS_s.refSamples.shape[0]))
outfile.write('Coverage of data points (TPR): %.3f, Requested >= %.3f\n' % (TPR, RS_s.M.alpha))
outfile.write('Coverage of reference points: (FPR) %.3f\n' % FPR)
outfile.write('Complexity: {}\n'.format(RS_s.complexity()))
outfile.write('Rules support: {}\n\n'.format(rule_str(rules1[0])))
outfile.write('Rules stats support: {}\n\n'.format(rules_stats(RS_s.rules, X_df_sample))) 


outfile.write('Rules overlap (clr): {}\n'.format(rule_str(rules1[1])))
outfile.write('Rules stats overlap: {}\n\n'.format(rules_stats(RS_o.rules, X_df_sample))) 
outfile.write('Number of Rules: {}\n'.format(np.sum([len(rule) for rule in rules1])))
outfile.write('Number of Literals: {}\n'.format(np.sum([len(rule_) for rule in rules1 for rule_ in rule])))
outfile.write('AUC between rules and base estimator (propensity): {}\n\n'.format(M.score_vs_base(X_df_sample)))


outfile.write('Rules overlap (knn): {}\n'.format(rule_str(rules2[1])))
outfile.write('Rules stats overlap: {}\n\n'.format(rules_stats(RS_o2.rules, X_df_sample))) 
outfile.write('Number of Rules: {}\n'.format(np.sum([len(rule) for rule in rules2])))
outfile.write('Number of Literals: {}\n'.format(np.sum([len(rule_) for rule in rules2 for rule_ in rule])))
outfile.write('AUC between rules and base estimator (propensity): {}\n\n'.format(M2.score_vs_base(X_df_sample)))

outfile.close()

In [10]:
from exps.supp_synthetic.synth_utils import compliance

def rules_stats(r_rules, df):

    rules = r_rules(transform=lambda a,b: b, fmt='%.1f')
    n_rules = float(len(rules))
    n_rules_literals = float(np.sum([len(rule) for rule in rules]))

    # Record more detailed rules information, e.g., proportion covered
    D = pd.concat([
        df,
        pd.DataFrame(np.ones_like(a_sample), columns=['support_set'])
        ], axis=1)
    Cs = compliance(D, rules)

    # This is everywhere, to be clear
    I1 = np.where(D['support_set'].values==1)[0]

    rule_stats = []
    for i in range(len(rules)):
        # Instances covered by rule
        d = {}
        d['rule'] = rules[i]
        d['n_covered'] = float(Cs[i][:,I1].prod(0).sum())
        d['p_covered'] = float(Cs[i][:,I1].prod(0).mean())
        rule_stats.append(d)
    return rule_stats

In [32]:
hyp_res = []

def find_hyp_config(hyp_res):
    max_auc = 0
    max_complexity = 0
    max_hyp = {}
    for res in hyp_res:
        if res['auc'] > max_auc:
            max_hyp = res
            max_auc = res['auc']
            max_complexity = res['complexity'][0] * res['complexity'][1]
            
    max_lambda1 = max_hyp['lambda1']
    
    candidates = []
    for res in hyp_res:
        if res['lambda1'] == max_lambda1:
            
            if res['complexity'][0] * res['complexity'][1] < max_complexity:
                candidates.append(res)
    
    max_c_auc = 0
    max_c = {}
    for c in candidates:
        if c['auc'] > max_c_auc:
            max_c = c
            max_c_auc = c['auc']
        
    return max_hyp, max_c

In [33]:
find_hyp_config(hyp_res)

({'auc': 0.8951770825586689,
  'complexity': (8, 59),
  'lambda0': 1e-05,
  'lambda1': 0.0001},
 {'auc': 0.8721470943239928,
  'complexity': (3, 16),
  'lambda0': 0.01,
  'lambda1': 0.0001})

In [11]:
with open('varencoding.txt', 'r') as f: 
    entire_doc = f.read()

encoding = entire_doc.split(';')

var_encoding = dict()
for var in encoding:
    var = var.strip()
    splits = var.split('\n')
    k = splits[0].replace('define', '').strip()
    vs = {}
    for i in range(1, len(splits)):
        v = splits[i].strip().split(" \"")
        vs.update({v[0]: v[1].replace('"', '')})
    
    var_encoding.update({k.lower(): vs})

var_list = pd.read_csv('data/encoding.csv')


def transcribe(rule_stats):
    for single_rset in rule_stats:
        single_rset_rule = single_rset['rule']
        p_covered = round(single_rset['p_covered'] * 100, 3)
        print(p_covered)
        for i in range(len(single_rset_rule)):
            rule = single_rset_rule[i]

            if '_' in rule[0]:
                var = rule[0].split("_")[0]
                level = rule[0].split("_")[1]

                var_str = var_list[var_list['var_name'] == var].label.values[0]
                level_str = var_encoding[var][level]

                rule_str = var_str + " is " + rule[1] + " \"" + level_str + "\""
                
            else:

                var = rule[0]
                var_str = var_list[var_list['var_name'] == var].label.values[0]
                rule_str = var_str + " " + rule[1] + " " +  str(round(rule[2]))
            if i != len(single_rset_rule) - 1:
                rule_str = rule_str + " and"
                    
#             print("& " + rule_str + " \\\\")
            print(rule_str)
        print('\n')

In [12]:
support_rule_stats = rules_stats(RS_s.rules, X_df_sample)
knn_rule_stats = rules_stats(RS_o2.rules, X_df_sample)

In [13]:
print("support")
transcribe(support_rule_stats)

print("overlap -knn")

transcribe(knn_rule_stats)

support
91.804
Type of toilet facility is not "Not a dejure resident" and
Household has: refrigerator is not "Not a dejure resident" and
Total children ever born > 0


1.363
Type of toilet facility is  "Not a dejure resident" and
Household has: refrigerator is  "Not a dejure resident" and
Total children ever born > 0 and
Sons at home <= 2 and
Cohabitation duration (grouped) is not "10-14" and
Fertility preference is not "Sterilized (respondent or partner)"


overlap -knn
98.973
Wealth index factor score combined (5 decimals) <= 290545.0 and
Wealth index factor score combined (5 decimals) > -188796.0




In [17]:
support = X_df[(~X_df["v116_97"].astype(bool) & ~X_df["v122_7"].astype(bool) & (X_df["v201"] > 0.500))| (X_df["v116_97"].astype(bool) & X_df["v122_7"].astype(bool) & (X_df["v201"] > 0.500) & (X_df["v202"] <= 2.500) & ~X_df["v513_3"].astype(bool) & ~X_df["v602_4"].astype(bool))]

propensity = X_df[((X_df["v191"] <= 290545.000) & (X_df["v191"] > -188796.000))]

print(len(support)/len(X_df))
print(len(propensity)/len(X_df))

X_overlap = pd.merge(support, propensity, how='inner')
print(len(X_overlap) / len(X_df))

0.9316693220038945
0.9897326960523987
0.9235262878385555


In [ ]:
import seaborn as sns
x = []
y = []
v = []
for r in results:
    x.append(r['lambda0'])
    y.append(r['lambda1'])
    
    v.append(r['auc'])

plot = sns.scatterplot(x=[str(X) for X in x], y=[str(Y) for Y in y], hue=v, hue_order= [0.5,0.6,0.7,0.8,0.9,1.0])
leg = plot.get_legend()
for t in leg.texts:
    t.set_text(float(t.get_text()[:4]))

plt.xlabel('λ0')
plt.ylabel('λ1')